# selenium으로 진행

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import psycopg2
from itertools import product
from time import sleep
import datetime

In [2]:
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.134/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\alsoj\.wdm\drivers\chromedriver\win32\103.0.5060.134]


In [3]:
BASE_URL = 'http://sbiznews.com/news/?menu=1&menuid=44&action=index'
# DETAIL_URL = 'https://ols.sbiz.or.kr/ols/man/SMAN052M/page.do?bltwtrSeq='
# https://ols.sbiz.or.kr/ols/man/SMAN052M/page.do?bltwtrSeq=4

In [4]:
browser.get(BASE_URL)

In [5]:
table = browser.find_element(by=By.XPATH, value='/html/body/table/tbody/tr[1]/td[2]/table/tbody/tr[2]/td/table/tbody/tr/td[2]/table/tbody/tr[2]/td[1]/table[2]/tbody/tr[2]/td/table/tbody')

In [6]:
rows = table.find_elements(by=By.TAG_NAME, value='tr')

In [12]:
today = datetime.datetime.now().strftime('%Y%m%d')
today = '20220725'

In [14]:
for row in rows:
    tds = row.find_elements(by=By.TAG_NAME, value='td')
    for td in tds:
        a_tag = td.find_elements(by=By.TAG_NAME, value='a')
        if len(a_tag) > 0:
            print(a_tag[0].get_attribute('href'), td.text)
        else :
            if today == td.text.replace("[","").replace("]","").replace("/","").strip():
                print(td.text.replace("[","").replace("]","").replace("/","").strip())

# a_tag[0].click()
# browser.switch_to.window(window_name=browser.window_handles[-1])

http://sbiznews.com/news/process.php?action=view&menuid=44&no=52916 금융위, 소상공인에 2년간 41.2조원 투입


IndexError: list index out of range

In [17]:
title = browser.find_element(by=By.CLASS_NAME, value='view_ltit').text
title

'금융위, 소상공인에 2년간 41.2조원 투입'

In [18]:
contents = browser.find_element(by=By.ID, value='ContentsLayertext').text
contents

'[창업경영신문 최환기자] 정부가 자영업자ㆍ소상공인의 유동성 공급, 경쟁력 강화, 재기지원을 위해 2년간 41.2조원 규모의 맞춤형 정책자금을 공급한다.\n금융위원회는 코로나19 대응 긴급ㆍ일시적 지원 조치가 종료되더라도 자영업자 등이 어려움을 겪지 않도록 선제적ㆍ종합적으로 대웅하기 위해 ‘맞춤형 종합지원 방안’을 마련했다고 24일 밝혔다.\n복합위기에 따른 유동성 공급, 거리두기 완화 이후 경쟁력 강화, 사업재기 등을 위해 맞춤형 자금을 공급할 필요가 있다는 것이 정부의 설명이다.\n이번 종합 지원방안 중 상환능력이 있는 차주에 대해 유동성 및 경쟁력제고 지원자금을 2년간 41조2000억 원 규모로 공급한다는 내용이 먼저 공개됐다. 이 정책자금은 ▲유동성 공급 10조5000억 원 ▲경쟁력 강화 29조7000억 원 ▲재기지원 1조 원으로 구성됐다.\n먼저, 유동성 공급 10조5000억 원은 특례보증, 금리부담 경감 등의 형식으로 지원된다. 신용보증기금과 기업은행이 코로나19 피해 등을 입은 자영업자ㆍ소상공인에 약 5조4000억 원 규모의 유동성을 지원하는데, 이중 신보가 25일부터 코로나로 직접적 피해를 입은 업체에 3조2500억 원 규모의 특례보증을 지원한다.\n지원대상은 소기업ㆍ소상공인과 매출액 50억 원 이하 중기업이다. 기업 당 운전자금 3억 원, 시설자금은 소요범위 내에서 지원한다. 보증료 0.5%포인트 감면, 보증비율 90%를 적용하며, 소진공ㆍ신보 컨설팅을 받은 업체는 보증료 0.1%포인트를 추가 우대한다.\n나머지 2조1000억 원은 기은과 신보가 매출감소ㆍ재무상황 악화 등으로 자금이 필요한 업체에 공급한다.\n금융위는 또 자영업자ㆍ소상공인의 경쟁력 강화를 위해 창업ㆍ사업 확장 및 설비투자 등을 위한 29조7000억 원 규모의 신규자금이 투입된다.\n기업은행에서는 신용도가 낮은 영세 소상공인 등을 위한 창업자금, 사업 내실화를 통한 설비투자 자금 등 18조 원을 공급한다. 신용보증기금은 창업ㆍ사업 확장, 서비스업 영위 등 자영업자ㆍ소상공인 특성에

In [21]:
url = browser.current_url
url

'http://sbiznews.com/news/?action=view&menuid=44&no=52916&page=&skey=&sword='

In [24]:
def insert_news(params):
    
    sql = """INSERT INTO tb_crawling_news_intrf(id, title, contents, gb, reg_dt, url, interface_dt)
             VALUES((select coalesce(max(id),0)+1 from tb_crawling_news_intrf), %s, %s, %s, %s, %s, now());"""
    conn = None

    try:
        conn = psycopg2.connect(host='58.120.227.138', dbname='kt_jalnagage',user='postgres',password='Wjdrlwjd1!',port=5432)
        cur = conn.cursor()
        
        cur.execute(sql, params)
        conn.commit()
        cur.close()
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [25]:
insert_news([title, contents, 'news', today, url])